In [1]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit

In [3]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [4]:
feature_list

{'SYN': ['Src Port',
  'Flow Duration',
  'Tot Fwd Pkts',
  'Tot Bwd Pkts',
  'TotLen Fwd Pkts',
  'TotLen Bwd Pkts',
  'Fwd Pkt Len Min',
  'Fwd Pkt Len Std',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Std',
  'Flow Byts/s',
  'Flow IAT Mean',
  'Flow IAT Std',
  'Flow IAT Max',
  'Flow IAT Min',
  'Fwd IAT Tot',
  'Fwd IAT Mean',
  'Fwd IAT Std',
  'Fwd IAT Max',
  'Fwd IAT Min',
  'Bwd IAT Mean',
  'Bwd IAT Std',
  'Bwd IAT Max',
  'Bwd IAT Min',
  'Bwd Pkts/s',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Len Std',
  'SYN Flag Cnt',
  'PSH Flag Cnt',
  'ACK Flag Cnt',
  'Pkt Size Avg',
  'Fwd Seg Size Avg',
  'Bwd Seg Size Avg',
  'Subflow Fwd Pkts',
  'Active Std',
  'Active Min',
  'Idle Mean',
  'Idle Std',
  'Idle Max',
  'Idle Min',
  'Label'],
 'HTTP': ['Dst Port',
  'Protocol',
  'Tot Fwd Pkts',
  'Tot Bwd Pkts',
  'TotLen Fwd Pkts',
  'TotLen Bwd Pkts',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Min',
  'Fwd Pkt Len Mean',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Min',
  'B

In [5]:
file_list={
"SYN":[ '../csvs\\dos-synflooding-1-dec.pcap_Flow.csv', '../csvs\\dos-synflooding-6-dec.pcap_Flow.csv',],
"HTTP":['../csvs\\mirai-httpflooding-4-dec.pcap_Flow.csv', '../csvs\\mirai-httpflooding-1-dec.pcap_Flow.csv'],
"ACK":['../csvs\\mirai-ackflooding-1-dec.pcap_Flow.csv','../csvs\\mirai-ackflooding-4-dec.pcap_Flow.csv'],
"UDP":[ '../csvs\\mirai-udpflooding-4-dec.pcap_Flow.csv','../csvs\\mirai-udpflooding-1-dec.pcap_Flow.csv'],
"ARP":[ '../csvs\\mitm-arpspoofing-6-dec.pcap_Flow.csv', '../csvs\\mitm-arpspoofing-1-dec.pcap_Flow.csv'],
"SP":['../csvs\\scan-hostport-6-dec.pcap_Flow.csv','../csvs\\scan-hostport-3-dec.pcap_Flow.csv'],
"BF":[ '../csvs\\mirai-hostbruteforce-4-dec.pcap_Flow.csv','../csvs\\mirai-hostbruteforce-1-dec.pcap_Flow.csv']
}

In [6]:
def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

In [7]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

### RandomizedSearchCV RF

In [8]:
lines=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]



for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))
    # use a full grid over all parameters
    param_grid = {"max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                  "n_estimators" : sp_randint(1, 200),
                  "max_features": sp_randint(1, 11),
                  "min_samples_split":sp_randint(2, 11),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    second=time()
    f1=[]
    clf=RandomForestClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(RandomForestClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("RF_HPO.csv",index=False)

final_parametres=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9914825015895742   0.688    0       


100%|██████████| 10/10 [02:39<00:00, 15.96s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9943676873288524   0.126    0       


100%|██████████| 10/10 [00:17<00:00,  1.74s/it]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.7850585657069448   0.445    0       


...

SP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.085    0       


100%|██████████| 10/10 [00:13<00:00,  1.32s/it]


BF
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.5303717369246379   0.175    0       


100%|██████████| 10/10 [00:40<00:00,  4.05s/it]

    bootst    criter      max_depth    max_features    min_samp_split    n_estimators        F1         Std    Time    No  Attack
--  --------  --------  -----------  --------------  ----------------  --------------  --------  ----------  ------  ----  --------
 0  True      entropy             5               8                 7             160  0.991483  0           19.316     1  SYN
 1  False     gini               18               8                 6              43  0.994931  0.00112733   1.412     8  HTTP
 2  True      entropy             4               8                 6             151  0.984909  0.00117981   8.258     6  ACK
 3  False     gini                9               7                 2             100  0.973457  0            1.241     7  UDP
 4  False     gini               22               2                 6              86  0.661237  0.0101059    1.488     7  ARP
 5  True      gini               14               2                10             143  1         0    

### RandomSearch CV Naive Bayes

In [9]:
from sklearn.naive_bayes import GaussianNB

In [10]:
lines=[['var_smoothing', "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
        'var_smoothing': np.logspace(0,-9, num=100),
    }

 
    second=time()
    f1=[]
    clf=GaussianNB()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(GaussianNB(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("NB_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    


SYN
default                             0.8602517092548031   0.018    0       


100%|██████████| 10/10 [00:02<00:00,  3.85it/s]


HTTP
default                             0.8477272914724974   0.003    0       


100%|██████████| 10/10 [00:00<00:00, 27.76it/s]


ACK
default                             0.6762772416658971   0.007    0       


...

SP
default                             0.8533824347392447   0.003    0       


100%|██████████| 10/10 [00:00<00:00, 27.03it/s]


BF
default                             0.3012227420848441   0.003    0       


100%|██████████| 10/10 [00:00<00:00, 32.26it/s]

      var_smoothing        F1          Std    Time    No  Attack
--  ---------------  --------  -----------  ------  ----  --------
 0      3.51119e-07  0.944144  0             0.258     0  SYN
 1      5.3367e-06   0.944187  0             0.259     1  SYN
 2      4.32876e-06  0.964114  0             0.275     2  SYN
 3      6.57933e-07  0.944144  0             0.252     3  SYN
 4      1e-07        0.902427  0             0.254     4  SYN
 5      3.51119e-07  0.944144  0             0.261     5  SYN
 6      3.51119e-06  0.973555  0             0.249     6  SYN
 7      4.32876e-05  0.874485  0             0.258     7  SYN
 8      2.84804e-08  0.89066   0             0.263     8  SYN
 9      1e-06        0.944144  0             0.262     9  SYN
10      5.3367e-09   0.834683  0             0.038     0  HTTP
11      1.87382e-09  0.844486  0             0.034     1  HTTP
12      4.32876e-08  0.821414  1.11022e-16   0.036     2  HTTP
13      2.84804e-09  0.834683  0             0.035     3  H

### RandomSearch CV SVM

In [11]:
lines = [["gamma", "C", "F1", "Std", "Time", "No", "Attack"]]

for j in file_list:
    print(j)

    df = pd.read_csv(file_list[j][0], usecols=feature_list[j])
    X_train = df.iloc[:, 0:-1]
    df["Label"] = df["Label"].astype("category")
    y_train = df["Label"].cat.codes

    df = pd.read_csv(file_list[j][1], usecols=feature_list[j])
    X_test = df.iloc[:, 0:-1]
    df["Label"] = df["Label"].astype("category")
    y_test = df["Label"].cat.codes

    X = np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [
        0 for _ in range(X_test.shape[0])
    ]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)

    param_grid = {"C": [0.001, 0.01, 0.1, 1, 10], "gamma": [0.001, 0.01, 0.1, 1]}

    second = time()
    f1 = []
    clf = svm.SVC()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict = clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
    f1 = sum(f1) / len(f1)
    # if f1>0.76:
    print("%-35s %-20s %-8s %-8s" % ("default", f1, round(time() - second, 3), ii))

    ######################################################################################################################
    for i in tqdm(range(1)):
        second = time()
        a, b, clf = run_random_search(svm.SVC(), param_grid, X, y)
        f1 = []
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict = clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
        f1_result = sum(f1) / len(f1)
        f1 = np.array(f1)
        stndtd = f1.std()
        temp = list(a.values())
        print("%-90s %-20s %-8s %-8s" % (a, f1_result, round(time() - second, 3), i))
        temp = temp + [f1_result, stndtd, round(time() - second, 3), i, j]
        lines.append(temp)

        # if f1>0.76:

results = pd.DataFrame(lines[1:], columns=lines[0])
results.to_csv("svm_HPO.csv", index=False)


print(tabulate(results, headers=list(results.columns)))


SYN
default                             0.7849428130587054   0.189    0       


  0%|          | 0/1 [02:55<?, ?it/s]


KeyboardInterrupt: 